In [1]:
import sqlite3
import pandas as pd

In [10]:
# Define the path to the primary database
primary_db_path = r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\USA\us_company_info.db'

# Connect to the primary database and load the data
conn = sqlite3.connect(primary_db_path)
primary_df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
conn.close()

print(len(primary_df))

11837


In [11]:
conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\DE\de_company_info.db')
deutsche_boerse_df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
print(len(deutsche_boerse_df))
conn.close()

conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\UK\uk_company_info.db')
lse_df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
print(len(lse_df))
conn.close()


3553
1803


In [5]:
# def create_merged_database(db_path):
#     conn = sqlite3.connect(db_path)
#     c = conn.cursor()
#     c.execute('''
#         CREATE TABLE IF NOT EXISTS companies (
#             id INTEGER PRIMARY KEY AUTOINCREMENT,
#             company_sec_cik TEXT,
#             company_name TEXT,
#             company_isin TEXT,
#             company_ticker TEXT,
#             company_exchange TEXT,
#             country TEXT,
#             industry TEXT,
#             sector TEXT,
#             longBusinessSummary TEXT,
#             embedding BLOB,
#             UNIQUE(company_name, company_ticker, company_isin)
#         )
#     ''')
#     conn.commit()
#     conn.close()

# create_merged_database('merged_company_info.db')

In [6]:
def insert_data(df, conn):
    c = conn.cursor()
    for _, row in df.iterrows():
        c.execute('''
            INSERT OR IGNORE INTO companies (
                company_sec_cik, company_name, company_isin, company_ticker, company_exchange,
                country, industry, sector, longBusinessSummary, embedding
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            row['company_sec_cik'], row['company_name'], row['company_isin'], row['company_ticker'], 
            row['company_exchange'], row['country'], row['industry'], row['sector'], 
            row['longBusinessSummary'], row.get('embedding')
        ))
    conn.commit()

In [7]:
def load_data(db_path):
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
    conn.close()
    return df

In [12]:
# Insert data into merged database
conn = sqlite3.connect("merged_company_info.db")
insert_data(primary_df, conn)
insert_data(deutsche_boerse_df, conn)
insert_data(lse_df, conn)
conn.close()

In [13]:
conn = sqlite3.connect("merged_company_info.db")
df = pd.read_sql_query("SELECT * FROM companies WHERE longBusinessSummary IS NOT NULL", conn)
print(len(df))
conn.close()

17191


In [1]:
import sqlite3

def delete_companies_with_na_or_null_longBusinessSummary():
    conn = sqlite3.connect('merged_company_info.db')
    c = conn.cursor()
    
    # Delete rows where longBusinessSummary is 'N/A'
    c.execute('''
              DELETE FROM companies
              WHERE longBusinessSummary = 'N/A'
              ''')
    
    # Delete rows where longBusinessSummary is NULL
    c.execute('''
              DELETE FROM companies
              WHERE longBusinessSummary IS NULL
              ''')
    
    conn.commit()
    conn.close()
    print("Deleted entries where longBusinessSummary is NULL or 'N/A'")

delete_companies_with_na_or_null_longBusinessSummary()

Deleted entries where longBusinessSummary is NULL or 'N/A'
